# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


In [4]:
spacex_df=pd.read_csv('spacex_launch_geo.csv')

In [5]:
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [20]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, launch_site in launch_sites_df.iterrows():
    folium.Circle(location=[launch_site['Lat'], launch_site['Long']], radius=1000,
                  color='#d35400', fill=True).add_child(folium.Popup(launch_site['Launch Site'])).add_to(site_map)
    folium.map.Marker(location=[launch_site['Lat'], launch_site['Long']],
                      icon=folium.Icon(color='red')).add_to(site_map)

site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [16]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    #Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [24]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [35]:
# Find coordinate of the closest coastline
# Example: Lat: 28.56367, Lon: -80.57163

# Define the launch site coordinates (example: VAFB SLC-4E)
launch_site_lat = launch_sites_df.loc[3, 'Lat']
launch_site_lon = launch_sites_df.loc[3, 'Long']

# Define the closest coastline coordinates (manually obtained using MousePosition)
coastline_lat = 34.63752  # Replace with actual latitude
coastline_lon = -120.62421  # Replace with actual longitude

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to coastline: {distance_coastline:.2f} km")

Distance to coastline: 1.34 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
distance_marker = folium.Marker(
	[coastline_lat, coastline_lon],
	icon=DivIcon(
		icon_size=(20,20),
		icon_anchor=(0,0),
		html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
		)
	)
# Add the marker to the map

site_map.add_child(distance_marker)

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [37]:
# Define the coordinates for the PolyLine
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [ ]:
# Define the coordinates of the closest city (manually obtained using MousePosition)
city_lat = 34.64362  # Replace with actual latitude
city_lon = -120.45781  # Replace with actual longitude

# Calculate the distance between the launch site and the closest city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
print(f"Distance to city: {distance_city:.2f} km")

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

# Add a marker for the closest city
city_marker = folium.Marker(
	[city_lat, city_lon],
	icon=DivIcon(
		icon_size=(20, 20),
		icon_anchor=(0, 0),
		html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_city),
	)
)
site_map.add_child(city_marker)
site_map

Distance to proximity point: 0.69 km


In [ ]:

# Draw a PolyLine between the launch site and the closest city
city_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]], weight=1)
site_map.add_child(city_line)

# Define the coordinates of the nearest railway (manually obtained using MousePosition)
railway_lat = 34.63668  # Replace with actual latitude
railway_lon = -120.62366  # Replace with actual longitude

# Calculate the distance between the launch site and the nearest railway
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
print(f"Distance to railway: {distance_railway:.2f} km")

# Add a marker for the nearest railway
railway_marker = folium.Marker(
	[railway_lat, railway_lon],
	icon=DivIcon(
		icon_size=(20, 20),
		icon_anchor=(0, 0),
		html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_railway),
	)
)
site_map.add_child(railway_marker)


In [ ]:

# Draw a PolyLine between the launch site and the nearest railway
railway_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]], weight=1)
site_map.add_child(railway_line)

# Define the coordinates of the nearest highway (manually obtained using MousePosition)
highway_lat = 34.68267  # Replace with actual latitude
highway_lon = -120.60382 # Replace with actual longitude

# Calculate the distance between the launch site and the nearest highway
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
print(f"Distance to highway: {distance_highway:.2f} km")

# Add a marker for the nearest highway
highway_marker = folium.Marker(
	[highway_lat, highway_lon],
	icon=DivIcon(
		icon_size=(20, 20),
		icon_anchor=(0, 0),
		html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_highway),
	)
)
site_map.add_child(highway_marker)

# Draw a PolyLine between the launch site and the nearest highway
highway_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]], weight=1)
site_map.add_child(highway_line)

#show map
print(site_map)

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


In [39]:
from dash import Dash, dcc, html

import plotly.express as px

# Initialize the Dash app
app = Dash(__name__)

# Create a scatter plot for launch records
fig = px.scatter(
	spacex_df,
	x="Lat",
	y="Long",
	color="class",
	hover_data=["Launch Site"],
	title="Launch Records: Success and Failure",
	labels={"class": "Launch Outcome"}
)

# Define the layout of the dashboard
app.layout = html.Div([
	html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
	dcc.Graph(
		id='launch-records-plot',
		figure=fig
	)
])

# Run the app
if __name__ == '__main__':
	app.run(debug=True)